# Mini-Project: Geospatial Data Science

We have collected the trajectories from Uber annotations done by 155 students working on the bonus project, and for each screenshot we keep a good trajectory. Those with no good trajectory has been filtered.

We obtain 3903 trajectories in total, which have been put in `trajectories_epsg_3857.txt`, where each row contains `img_id` and corresponding trajectory in EPSG:3857

Example:\
`Participant2_2019_March2019_Shot0146.jpg:-9665838.908 3954722.303,-9667308.043 3953792.864,-9667400.48 3953250.484,-9666948.894 3952362.957,-9666948.894 3952362.957`

In [21]:
#vjvanka.19@gmail.com google drive is used
#hvanka@uab.edu
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Downland and Loading

In [22]:
# get trajectories file from Github repo
import requests

url = 'https://github.com/jalal1/geospatial_data_science_mini/raw/main/trajectories_epsg_3857.txt'
req = requests.get(url, allow_redirects=True)

open('trajectories_epsg_3857.txt', 'wb').write(req.content)

773390

In [23]:
!head -n 1 trajectories_epsg_3857.txt

Participant2_2019_March2019_Shot0146.jpg:-9665838.908 3954722.303,-9667308.043 3953792.864,-9667400.48 3953250.484,-9666948.894 3952362.957,-9666948.894 3952362.957


In [24]:
# read all trajectories from file into dictionary 'img2trajectory'
import os
img2trajectory = {} # {img_id:trajectory} 
n_line = 0

with open('trajectories_epsg_3857.txt', 'r') as f:
    for line in f:
      n_line += 1
      line_values = line.split(":")
      img_id = line_values[0]
      traj = line_values[1]
      img2trajectory[img_id] = []
      for x in traj.split(','):
        lat, lon = x.split(" ")
        img2trajectory[img_id].append((float(lat), float(lon)))

print("Number of trajectories: ", len(img2trajectory))

Number of trajectories:  3903


In [25]:
img2trajectory['Participant2_2019_March2019_Shot0146.jpg']

[(-9665838.908, 3954722.303),
 (-9667308.043, 3953792.864),
 (-9667400.48, 3953250.484),
 (-9666948.894, 3952362.957),
 (-9666948.894, 3952362.957)]

### Using PyProj package

Cartographic projections and coordinate transformations library, https://pyproj4.github.io/pyproj/stable/#

We may use PyProj to transform one coordinate system to another.

In [26]:
! pip install pyproj

In [27]:
# Example: Convert from EPSG:3857 to EPSG:4326, check here: https://epsg.io/ for more details.
from pyproj import Transformer

def convert_epsg(source_epsg, dist_epsg, x, y):
    transformer = Transformer.from_crs(source_epsg, dist_epsg)
    x, y = transformer.transform(x, y)
    return x, y

input_coords = (-9662852.767, 3962856.202)
convert_epsg('epsg:3857', 'epsg:4326', input_coords[0], input_coords[1])

(33.507460522863106, -86.8028832879271)

Remember in the bouns project we used https://www.georeferencer.com/ to get the coordinates. The format was epsg:3857

We need to convert to GPS coordinates in epsg:4326

In [28]:
import os
from glob import glob
import cv2
import matplotlib.pyplot as plt
# TODO: 
# 1. using the images assigned to you in the bouns project, get image names and save into a new list.
# --- You can find your assignment from here: https://uab.instructure.com/files/68292984/download?download_frd=1
# 2. using image names, get trajectories from img2trajectory dictionary
# 3. Convert all trajectories from epsg:3857 to epsg:4326 and save it to new dictionary to be used later

img2trajectory_4326 = {}
#img2trajectory = {}
# !!! YOUR CODE !!!
#url = 'https://uab.instructure.com/files/68292984/download?download_frd=1'
#req = requests.get(url, allow_redirects=True)

#open('trajectories_epsg_3857.txt', 'wb').write(req.content)

path  = '/content/drive/MyDrive/fds/HVANKA'
imageshvanka= os.listdir(path)
imageshvanka


# {img_id:trajectory} 
#n_line = 0

for image_ins in imageshvanka:
    trajectories = img2trajectory[image_ins]
    converthvanka_4326 = [convert_epsg('epsg:3857', 'epsg:4326', idd[0], idd[1]) for idd in trajectories]
    img2trajectory_4326[image_ins] = converthvanka_4326



### Visualizing a trajectory using Folium

Folium allows us to overlay plots on top of a map.

In [29]:
! pip install folium

In [30]:
# plot a trajectory
import folium
import numpy as np

img_id = imageshvanka[16] # take the 3rd image and plot, you can change it

traj = img2trajectory_4326[img_id]

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

map

In [31]:
# at each point on the trajectory, show a marker to check the order of points. 
import time
from IPython.display import clear_output

pos = 0
while pos < len(traj):
    folium.Marker(traj[pos]).add_to(map)
    pos += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.5)

When we were checking if an annotation is good, we did not check the point orders, so some could be bad, which we need your help to pick them out.

Each image was assigned to 3 students, so the trajectory you see we picked may not be yours

In [32]:
# Below is an example of one image with an issue in the trajectory. We need to find such cases
img_id = 'uber_imgs_Participant6_September2021_Shot0148.jpg'

# In case you want to check more images. Sample a random image and check the trajectory
# import random
# img_id = random.choice(list(img2trajectory.keys()))

# convert coordinates
traj = []
for point in img2trajectory[img_id]:
  lat, lon = convert_epsg('epsg:3857', 'epsg:4326', point[0], point[1])
  traj.append((lat, lon))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

pos = 0
while pos < len(traj):
    folium.Marker(traj[pos]).add_to(map)
    pos += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.5)

In [37]:
# TODO: 
# Using your assigned pool of 59-60 images, check them one by one (like the previous cell) to pick out those bad trajectories like above.
# Write the results to an output file
# The output files should contain lines of the format: img_id,good/bad.
# Good means the trajectory points are moving in the correct order of the underlying trajectory, not jumping here and there

# output file example lines: 
# Participant5_2019_March2019_Shot0054.jpg,good
# Participant2_2019_October2019_Shot0270.jpg,bad
# ...

# Please name your file with your BLAZER ID only! Do not add suffix like .txt

# !!! YOUR CODE !!!
#import random
#img_id = random.choice(list(img2trajectory.keys()))
image_ins = 'Participant2_2019_March2019_Shot0135.jpg'
#img_id = imageshvanka[16]


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [38]:
image_ins = 'Participant2_2019_May2019_Shot0137.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [39]:
image_ins = 'Participant2_2019_November2019_Shot0121.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [40]:
image_ins = 'Participant2_2019_November2019_Shot0194.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [41]:
image_ins = 'Participant2_2019_November2019_Shot0263.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [42]:
image_ins = 'Participant2_2019_October2019_Shot0182.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [44]:
image_ins = 'Participant2_2019_October2019_Shot0253.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [45]:
image_ins = 'Participant2_2019_October2019_Shot0328.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [46]:
image_ins = 'Participant2_2019_September2019_Shot0139.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [47]:
image_ins = 'Participant2_2019_September2019_Shot0212.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [48]:
image_ins = 'Participant2_2021_April2021_Shot0218.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [49]:
image_ins = 'Participant2_2021_March2021_Shot0135.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [51]:
image_ins = 'Participant2_2021_November2021_Shot0148.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [52]:
image_ins = 'Participant2_2021_October2021_Shot0119.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [53]:
image_ins = 'Participant2_2021_September2021_Shot0366.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [54]:
image_ins = 'Participant2_2021_September2021_Shot0461.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [55]:
image_ins = 'Participant3_2019_April2019_Shot0224.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [56]:
image_ins = 'Participant3_2019_April2019_Shot0296.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [57]:
image_ins = 'Participant3_2019_March2019_Shot0016.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [58]:
image_ins = 'Participant3_2019_March2019_Shot0083.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [59]:
image_ins = 'Participant3_2019_March2019_Shot0153.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [60]:
image_ins = 'Participant3_2019_May2019_Shot0376.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [61]:
image_ins = 'Participant3_2019_November2019_Shot0671.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [62]:
image_ins = 'Participant3_2019_November2019_Shot0743.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [63]:
image_ins = 'Participant3_2019_October2019_Shot0522.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [64]:
image_ins = 'Participant3_2019_October2019_Shot0595.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [65]:
image_ins = 'Participant3_2019_September2019_Shot0446.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [66]:
image_ins = 'Participant3_2021_April2021_Shot0031.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [67]:
image_ins = 'Participant4_2019_November2019_Shot0002.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [68]:
image_ins = 'Participant4_2019_October2019_Shot0131.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [69]:
image_ins = 'Participant4_2019_September2019_Shot0061.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [70]:
image_ins = 'Participant5_2019_April2019_Shot0012.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [71]:
image_ins = 'Participant5_2019_March2019_Shot0027.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [72]:
image_ins = 'Participant5_2019_May2019_Shot0030.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [73]:
image_ins = 'Participant5_2021_May2021_Shot0104.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [74]:
image_ins = 'Participant5_2021_October2021_Shot0067.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [75]:
image_ins = 'Participant5_2021_October2021_Shot0136.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [76]:
image_ins = 'Participant5_2021_September2021_Shot0007(2).jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [77]:
image_ins = 'Participant7_2021_November2021_Shot0018.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [78]:
image_ins = 'Participant7_2021_November2021_Shot0085.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [79]:
image_ins = 'Participant8_2021_November2021_Shot0040.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [80]:
image_ins = 'Participant8_2021_November2021_Shot0107.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [81]:
image_ins = 'Participant8_2021_October2021_Shot0217.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [82]:
image_ins = 'Participant8_2021_October2021_Shot0288.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [83]:
image_ins = 'Participant8_2021_September2021_Shot0001.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [84]:
image_ins = 'Participant8_2021_September2021_Shot0075.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [85]:
image_ins = 'Participant8_2021_September2021_Shot0147.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [86]:
image_ins = 'uber_imgs_Participant1_July2021_IMG_3759.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [87]:
image_ins = 'uber_imgs_Participant1_June2021_IMG_3726.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [88]:
image_ins = 'uber_imgs_Participant6_April2021_Shot0054.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [89]:
image_ins = 'uber_imgs_Participant6_March2021_Shot0003.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [90]:
image_ins = 'uber_imgs_Participant6_March2021_Shot0071.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [91]:
image_ins = 'uber_imgs_Participant6_March2021_Shot0142.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [92]:
image_ins = 'uber_imgs_Participant6_November2021_Shot0017.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [93]:
image_ins = 'uber_imgs_Participant6_November2021_Shot0082.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [94]:
image_ins = 'uber_imgs_Participant6_October2021_Shot0057.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [95]:
image_ins = 'uber_imgs_Participant6_October2021_Shot0124.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [96]:
image_ins = 'uber_imgs_Participant6_September2021_Shot0080.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

In [98]:
image_ins = 'uber_imgs_Participant6_September2021_Shot0148.jpg'


traj = []
for iddd in img2trajectory[image_ins]:
  x, y = convert_epsg('epsg:3857', 'epsg:4326', iddd[0], iddd[1])
  traj.append((x, y))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

count = 0

while count < len(traj):
    folium.Marker(traj[count]).add_to(map)
    count += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.2)

### Fast map matching (FMM)*

Reference: Can Yang & Gyozo Gidofalvi (2018) Fast map matching, an algorithm
integrating hidden Markov model with precomputation, International Journal of Geographical Information Science, 32:3, 547-570, DOI: 10.1080/13658816.2017.1400548

https://fmm-wiki.github.io/

As you may have noticed, the trajectories are not aligned with the underlying road network. i.e., a line between two consecutive points is a straight line and does not follow the actual roads.

To do match those points to the road network, and add back intermediate points, we use FMM. First, we need to install it.

**Install Prerequisites**

In [99]:
### TODO ###

# install FMM "requirements" following https://fmm-wiki.github.io/docs/installation
# You need to choose the right OS on that page
# For example, on Ubuntu you can run
!sudo apt-get install libboost-dev libboost-serialization-dev gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-dev is already the newest version (1.65.1.0ubuntu1).
make is already the newest version (4.1-9.1ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
gdal-bin is already the newest version (2.2.3+dfsg-2).
libboost-serialization-dev is already the newest version (1.65.1.0ubuntu1).
libgdal-dev is already the newest version (2.2.3+dfsg-2).
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libexpat1-dev is already the newest version (2.2.5-3ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


**Install FMM**

In [100]:
!git clone https://github.com/cyang-kth/fmm.git

Cloning into 'fmm'...
remote: Enumerating objects: 5162, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5162 (delta 14), reused 35 (delta 10), pack-reused 5119
Receiving objects: 100% (5162/5162), 15.33 MiB | 28.81 MiB/s, done.
Resolving deltas: 100% (3062/3062), done.


In [101]:
# !!!! This could take ~ 6 mintues !!!!
# it may not work and requires for password, etc.
# in this case, go to your folder using console to compile and install FMM

import os
# change working directory
os.chdir("fmm")

if not os.path.exists('build'):
  os.mkdir('build')
# ! mkdir build
os.chdir("build")
# ! cd build
! cmake ..
! make -j4
! sudo make install

-- CMAKE version 3.12.0
-- Set CMP0074 state to NEW
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- 
No conda environment found in PATH!
PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin

-- Could NOT find Conda (missing: CONDA_PREFIX) 
-- Non conda exist, search library in default path
-- Found GDAL: /usr/lib/libgdal.so (Required is at l

In [102]:
# Verfication of installation
!fmm

[info][fmm_app_config.cpp:49 ] Start reading FMM configuration from arguments
fmm argument lists:
--ubodt (required) <string>: Ubodt file name
--network (required) <string>: Network file name
--network_id (optional) <string>: Network id name (id)
--source (optional) <string>: Network source name (source)
--target (optional) <string>: Network target name (target)
--gps (required) <string>: GPS file name
--gps_id (optional) <string>: GPS id name (id)
--gps_x (optional) <string>: GPS x name (x)
--gps_y (optional) <string>: GPS y name (y)
--gps_timestamp (optional) <string>: GPS timestamp name (timestamp)
--gps_geom (optional) <string>: GPS geometry name (geom)
--gps_point (optional): if specified read input data as gps point, otherwise (default) read input data as trajectory
--output (required) <string>: Output file name
--output_fields (optional) <string>: Output fields
  opath,cpath,tpath,mgeom,pgeom,
  offset,error,spdist,tp,ep,length,duration,speed,all
-k/--candidates (optional) <int>

In [103]:
# Change to the parent folder which contains fmm_test.py
if os.getcwd() != "/content/fmm/example/python":
  os.chdir("/content/fmm/example/python")
os.system('python fmm_test.py')

256

In [104]:
# Install osmnx package to prepare the network, that will be used as input to FMM
!pip install osmnx

     |████████████████████████████████| 95 kB 1.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 1.0 MB 39.6 MB/s 
     |████████████████████████████████| 1.0 MB 10.9 MB/s 
     |████████████████████████████████| 11.2 MB 27.8 MB/s 
     |████████████████████████████████| 16.7 MB 43.1 MB/s 
     |████████████████████████████████| 930 kB 40.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incomp

## Steps to run FMM after installation: 
1- Prepare the network using osmnx \
2- Run the match(...) function

In [105]:
# !!!! This could take ~ 2 mintues !!!!

import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import numpy as np
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

# !!! Download by a bounding box # !!!
# -------------------------------------------------
bounds = (-86.9671,-86.5901,33.3472,33.6598)
# -------------------------------------------------
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./network-new')
print("--- %s seconds ---" % (time.time() - start_time))

osmnx version 1.1.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


--- 29.292325496673584 seconds ---


In [106]:
# copy network folder to /content/fmm/example/python/network-new
!mv '/content/network-new' '/content/fmm/example/python/network-new'

mv: cannot stat '/content/network-new': No such file or directory


In [107]:
# match(...) function

def match(path,points):
  network = Network(path,"fid","u","v")
  graph = NetworkGraph(network)
  print (graph.get_num_vertices())
  model = STMATCH(network,graph)
  wkt  = str(points)
  config = STMATCHConfig()
  config.k = 4
  config.gps_error = 0.5
  config.radius = 0.4
  config.vmax = 30;
  config.factor =1.5
  result = model.match_wkt(wkt,config)
  print (type(result))
  # print ("Opath ",list(result.opath))
  # print ("Cpath ",list(result.cpath))
  # print ("WKT ",result.mgeom.export_wkt())
  return result.mgeom.export_wkt()

In [108]:
def match_trajs(network_path, traj):
  """
  match [traj] on [network_path]. [traj] should be [(lon1, lat1), (lon2, lat2), ...]
  Output: matched traj as "LINESTRING(-86.797211 33.499194,-86.7973..."
  """
  from shapely.geometry import LineString
  # import geopandas !!

  traj_ = LineString(traj)
  return match(network_path, traj_)

In [109]:
# get a random trajectory
import random
img_id = random.choice(list(img2trajectory_4326.keys()))
traj = img2trajectory_4326[img_id]
print("Original Trajectory: ", traj)
# swap (lat, lon) --> (lon, lat) as FMM requires lon before lat
traj_m = [(sub[1], sub[0]) for sub in traj]
fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
# remove LINESTRING and extra parentheses
fmm_traj = fmm_traj.split("LINESTRING(")[1]
fmm_traj = fmm_traj.split(")")[0]
print("Matched Trajectory: ", fmm_traj)

Original Trajectory:  [(33.58653794492783, -86.75584939509584), (33.591522227053275, -86.69705340210882), (33.639726043268574, -86.66135430780216)]
32332
<class 'fmm.PyMatchResult'>
Matched Trajectory:  -86.754681 33.585329,-86.754364 33.585636,-86.75405 33.585931,-86.753741 33.586189,-86.753472 33.586366,-86.753345 33.586437,-86.753112 33.586567,-86.752993 33.586617,-86.752749 33.586719,-86.752322 33.586868,-86.752253 33.586892,-86.749687 33.587705,-86.749044 33.587913,-86.748786 33.588021,-86.748543 33.588123,-86.748447 33.58817,-86.748074 33.588368,-86.747659 33.588629,-86.747352 33.588821,-86.746608 33.589288,-86.745645 33.589893,-86.745135 33.590115,-86.744636 33.590253,-86.744084 33.590311,-86.743656 33.590315,-86.742529 33.590294,-86.741562 33.590276,-86.740304 33.590253,-86.739962 33.590247,-86.739583 33.590231,-86.739208 33.590216,-86.739064 33.59021,-86.738549 33.590152,-86.737408 33.589987,-86.737194 33.589956,-86.736455 33.589938,-86.735936 33.589934,-86.735369 33.589939,-8

In [110]:
# plot both trajctories, the original and the matched one.

matched_traj = []
for c in fmm_traj.split(","):
    lon, lat = c.split(" ")
    matched_traj.append((float(lat), float(lon)))

center = np.asarray(matched_traj[0]) + np.asarray(matched_traj[-1])
center /= 2

start_level = 13
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)
folium.PolyLine(matched_traj, color='green', weight=5, opacity=0.8).add_to(map)

map

We can also visualize the trajectories using Folium HeatMap

In [111]:
from folium.plugins import HeatMap

map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
HeatMap(matched_traj, blur=5, radius=5).add_to(map)
map

In [112]:
# to understand the density of trips, we can decompose trajectories into points and plot
# decompose your trajectories and add to a new list. Then use Heatmap to plot

# !!! YOUR CODE !!!
matched_traj = []
for img_id in imageshvanka:
    traj = img2trajectory_4326[img_id]
   
    traj_m = [(sub[1], sub[0]) for sub in traj]
    fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
    
    fmm_traj = fmm_traj.split("LINESTRING(")[1]
    fmm_traj = fmm_traj.split(")")[0]
   

    for i in fmm_traj.split(","):
        matched_traj.append((float(lat), float(lon)))

    center = np.asarray(matched_traj[0]) + np.asarray(matched_traj[-1])
    center /= 2

    start_level = 12
    map = folium.Map(zoom_start=start_level, location=center)
    folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)
    folium.PolyLine(matched_traj, color='green', weight=5, opacity=0.8).add_to(map)

    map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
    HeatMap(matched_traj, blur=5, radius=5).add_to(map)
display(map)

32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class '

In [113]:
# The End